### Install dependencies

In [ ]:
%%bash
# git clone https://github.com/KaiyangZhou/Dassl.pytorch.git
cd Dassl.pytorch/
# Install dependencies
pip install -r requirements.txt
# Install this library (no need to re-build if the source code is modified)
python setup.py develop
cd ..
# Install dependencies of MaPLe
pip install -r requirements.txt

## generate ground truth instance labels

In [44]:
import os
import numpy as np
import torch
import glob

os.makedirs("../../dataset/data/replica_features/train", exist_ok=True)
os.makedirs("../../dataset/data/replica_features/val", exist_ok=True)
os.makedirs("../../dataset/data/replica_features/test", exist_ok=True)
train_test_dict = {
    "office0": "train",
    "office1": "train",
    "office2": "train",
    "office3": "val",
    "office4": "test",
    "room0": "train",
    "room1": "val",
    "room2": "test",
}

gt_masks = glob.glob("../../dataset/OpenYOLO3D/output/replica/replica_ground_truth_masks/*.pt")
# gt_instances = glob.glob("dataset/replica_split/ground_truth/*.txt")
for gt_mask_path in gt_masks:
    gt_mask = torch.load(gt_mask_path)[0]
    gt_instance_id_path = gt_mask_path.replace("OpenYOLO3D/output/replica/replica_ground_truth_masks/", "data/replica_split/ground_truth/").replace(".pt", ".txt")
    gt_instance_id = np.loadtxt(gt_instance_id_path, dtype='int')
    assert gt_mask.shape[0] == gt_instance_id.shape[0]
    
    labels = np.zeros(gt_mask.shape[1])
    for instance_id in range(gt_mask.shape[1]):
        mask_points = gt_mask[:, instance_id].bool()
        instance_label = gt_instance_id[mask_points][0]
        semantics_label = instance_label // 1000
        labels[instance_id] = semantics_label
    scene_name = os.path.splitext(os.path.basename(gt_mask_path))[0]
    split = train_test_dict[scene_name]
    output_path = f"../../dataset/data/replica_features/{split}/{scene_name}_labels"
    np.save(output_path, labels)

## generate instance feature (openscene features and ground truth masks)

In [60]:
# ADJUST PATH!!!
openscene_feature_path = glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/*_features.npy")
openscene_feature_path.extend(glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/*_features.npy"))
openscene_feature_path.extend(glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/*_features.npy"))

for feature_path in openscene_feature_path:
    print(feature_path)
    scene_name = os.path.splitext(os.path.basename(feature_path))[0]
    scene_name = scene_name.replace("_features", "")
    gt_mask_path = f"../../dataset/OpenYOLO3D/output/replica/replica_ground_truth_masks/{scene_name}.pt"

    point_features = np.load(feature_path)
    gt_mask = torch.load(gt_mask_path)[0]
    instance_features = np.zeros((gt_mask.shape[1], point_features.shape[1]))
    for instance_id in range(gt_mask.shape[1]):
        mask_points = gt_mask[:, instance_id].bool()
        mask_features = point_features[mask_points, :]
        instance_feature = np.mean(mask_features, axis=0)  # aggregate
        instance_features[instance_id, :] = instance_feature
    split = train_test_dict[scene_name]
    output_path = f"../../dataset/data/replica_features/{split}/{scene_name}_features"
    np.save(output_path, instance_features)
    

../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office1_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/room0_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office0_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office2_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/room1_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/office3_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/office4_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/room2_features.npy


## Prompt Learning

In [1]:
# %%bash
# bash scripts/maple_prompt_scene/base2new_train_maple_prompt_scene.sh replica 1 ../../dataset/data/

Run this job and save the output to output/base2new/train_base/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1
Error while terminating subprocess (pid=2879446): 


In [3]:
# %%bash
# bash scripts/maple_prompt_scene/base2new_test_maple_prompt_scene.sh replica 1 ../../dataset/data/

Evaluating model
Runing the first phase job and save the output to output/base2new/test_new/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Error while terminating subprocess (pid=2879966): 


In [63]:
os.makedirs("../../dataset/data/replica_features/split_fewshot", exist_ok=True)

In [65]:
%%bash
bash scripts/maple_prompt_scene/xd_train_maple_prompt_scene.sh replica 1 ../../dataset/data/

Run this job and save the output to output/replica/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_1000000shots/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: False
head: 
load_epoch: None
model_dir: 
no_train: False
opts: ['DATASET.NUM_SHOTS', '1000000']
output_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_1000000shots/seed1
resume: 
root: ../../dataset/data/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATASET:
  ALL_AS_UNLABELED: False
  CIFAR_C_LEVEL: 1
  CIFAR_C_TYPE: 
  N

val: [<datasets.replica.Datum_feature object at 0x7ff5b2cd1600>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1660>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1f00>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1840>, <datasets.replica.Datum_feature object at 0x7ff5b2cd18d0>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1780>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1900>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1cc0>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1b40>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1ab0>, <datasets.replica.Datum_feature object at 0x7ff5b2cd19c0>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1a20>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1a80>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1bd0>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1c30>, <datasets.replica.Datum_feature object at 0x7ff5b2cd1c90>, <datasets.replica.Datum_feature object at 0x7ff5b2

-----------------------------------------------------------
SUBSAMPLE ALL CLASSES!
-----------------------------------------------------------
train: [<datasets.replica.Datum_feature object at 0x7ff5b2ce2950>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2980>, <datasets.replica.Datum_feature object at 0x7ff5b2ce29e0>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2a40>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2aa0>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2b00>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2b60>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2bc0>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2c20>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2c80>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2ce0>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2d40>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2da0>, <datasets.replica.Datum_feature object at 0x7ff5b2ce2e00>, <datasets.replica.Datum

val: [<datasets.replica.Datum_feature object at 0x7ff5b2cf6c20>, <datasets.replica.Datum_feature object at 0x7ff5b2cf6c80>, <datasets.replica.Datum_feature object at 0x7ff5b2cf6ce0>, <datasets.replica.Datum_feature object at 0x7ff5b2cf6d40>, <datasets.replica.Datum_feature object at 0x7ff5b2cf6da0>, <datasets.replica.Datum_feature object at 0x7ff5b2cf6e00>, <datasets.replica.Datum_feature object at 0x7ff5b2cf6e60>, <datasets.replica.Datum_feature object at 0x7ff5b2cf6ec0>, <datasets.replica.Datum_feature object at 0x7ff5b2cf6f20>, <datasets.replica.Datum_feature object at 0x7ff5b2cf6f80>, <datasets.replica.Datum_feature object at 0x7ff5b2cf6fe0>, <datasets.replica.Datum_feature object at 0x7ff5b2cf7040>, <datasets.replica.Datum_feature object at 0x7ff5b2cf70a0>, <datasets.replica.Datum_feature object at 0x7ff5b2cf7100>, <datasets.replica.Datum_feature object at 0x7ff5b2cf7160>, <datasets.replica.Datum_feature object at 0x7ff5b2cf71c0>, <datasets.replica.Datum_feature object at 0x7ff5b2

-----------------------------------------------------------
Label range: 0-47
Unique labels: Counter({45: 18, 22: 16, 17: 14, 5: 11, 38: 11, 13: 10, 46: 10, 44: 7, 6: 7, 3: 6, 25: 6, 4: 5, 35: 5, 36: 5, 20: 4, 27: 4, 37: 4, 7: 3, 40: 3, 19: 2, 28: 2, 21: 2, 30: 2, 31: 2, 47: 2, 10: 2, 14: 2, 39: 2, 41: 2, 15: 1, 18: 1, 23: 1, 0: 1, 11: 1, 12: 1, 26: 1, 32: 1, 43: 1})
Label range: 0-47
Unique labels: Counter({22: 12, 13: 10, 46: 9, 5: 8, 28: 7, 45: 7, 37: 4, 44: 4, 17: 4, 6: 3, 29: 3, 38: 3, 4: 2, 20: 2, 24: 2, 25: 2, 2: 2, 3: 2, 7: 2, 39: 2, 0: 1, 1: 1, 11: 1, 16: 1, 21: 1, 26: 1, 31: 1, 43: 1, 47: 1, 8: 1, 10: 1, 14: 1, 19: 1, 35: 1, 40: 1, 42: 1})
Label range: 2-47
Unique labels: Counter({13: 18, 22: 8, 45: 7, 5: 6, 44: 5, 46: 5, 8: 5, 43: 5, 2: 2, 3: 2, 20: 2, 25: 2, 38: 2, 21: 2, 31: 2, 33: 2, 10: 1, 14: 1, 41: 1, 7: 1, 9: 1, 34: 1, 37: 1, 47: 1})
Building transform_train
+ random resized crop (size=(224, 224), scale=(0.08, 1.0))
+ random flip
+ to torch tensor of range [0, 1]
+ no

100%|██████████| 1/1 [00:00<00:00,  1.45it/s]


output: tensor([[ 4.4688,  5.8164,  4.8086,  ...,  7.3281,  7.4805,  4.0977],
        [ 5.5742,  6.9219,  5.9414,  ...,  8.4219,  8.5781,  5.1641],
        [ 4.9922,  6.0742,  5.3750,  ...,  7.4219,  7.5273,  4.6953],
        ...,
        [ 7.1680,  8.0547,  7.3828,  ...,  9.9453,  9.5000,  6.5625],
        [ 8.5078,  9.3906,  8.7109,  ..., 11.3203, 10.9062,  7.8828],
        [ 2.6543,  4.1367,  2.7383,  ...,  4.8281,  5.0078,  2.5039]],
       device='cuda:0', dtype=torch.float16)
label: tensor([ 2,  2,  3,  3, 10, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 20, 22,
        22, 22, 22, 25, 25, 38, 41, 44, 45, 45, 45, 46, 46,  5,  5,  5,  5,  5,
         5,  7,  8,  8,  8,  8,  8,  9, 13, 13, 13, 13, 13, 13, 13, 13, 20, 21,
        21, 22, 22, 22, 22, 31, 31, 33, 33, 34, 37, 38, 43, 43, 43, 43, 43, 44,
        44, 44, 44, 45, 45, 45, 45, 46, 46, 46, 47], device='cuda:0')
=> result
* total: 83
* correct: 26
* accuracy: 31.3%
* error: 68.7%
* macro_f1: 5.6%
Elapsed: 0:00:09


In [69]:
%%bash
bash scripts/maple_prompt_scene/xd_test_maple_prompt_scene.sh replica 1 ../../dataset/data/

Run this job and save the output to output/evaluation/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_1000000shots/replica/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: True
head: 
load_epoch: 2
model_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_1000000shots/seed1
no_train: False
opts: []
output_dir: output/evaluation/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_1000000shots/replica/seed1
resume: 
root: ../../dataset/data/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATAS

val: [<datasets.replica.Datum_feature object at 0x7fee38bdd7b0>, <datasets.replica.Datum_feature object at 0x7fee38bdd810>, <datasets.replica.Datum_feature object at 0x7fee38bde0b0>, <datasets.replica.Datum_feature object at 0x7fee38bdd9f0>, <datasets.replica.Datum_feature object at 0x7fee38bdda80>, <datasets.replica.Datum_feature object at 0x7fee38bdd930>, <datasets.replica.Datum_feature object at 0x7fee38bddab0>, <datasets.replica.Datum_feature object at 0x7fee38bdde70>, <datasets.replica.Datum_feature object at 0x7fee38bddcf0>, <datasets.replica.Datum_feature object at 0x7fee38bddc60>, <datasets.replica.Datum_feature object at 0x7fee38bddb70>, <datasets.replica.Datum_feature object at 0x7fee38bddbd0>, <datasets.replica.Datum_feature object at 0x7fee38bddc30>, <datasets.replica.Datum_feature object at 0x7fee38bddd80>, <datasets.replica.Datum_feature object at 0x7fee38bddde0>, <datasets.replica.Datum_feature object at 0x7fee38bdde40>, <datasets.replica.Datum_feature object at 0x7fee38

-----------------------------------------------------------
SUBSAMPLE ALL CLASSES!
-----------------------------------------------------------
train: [<datasets.replica.Datum_feature object at 0x7fee38bee590>, <datasets.replica.Datum_feature object at 0x7fee38bee5f0>, <datasets.replica.Datum_feature object at 0x7fee38bee650>, <datasets.replica.Datum_feature object at 0x7fee38bee6b0>, <datasets.replica.Datum_feature object at 0x7fee38bee710>, <datasets.replica.Datum_feature object at 0x7fee38bee770>, <datasets.replica.Datum_feature object at 0x7fee38bee7d0>, <datasets.replica.Datum_feature object at 0x7fee38bee830>, <datasets.replica.Datum_feature object at 0x7fee38bee890>, <datasets.replica.Datum_feature object at 0x7fee38bee8f0>, <datasets.replica.Datum_feature object at 0x7fee38bee950>, <datasets.replica.Datum_feature object at 0x7fee38bee9b0>, <datasets.replica.Datum_feature object at 0x7fee38beea10>, <datasets.replica.Datum_feature object at 0x7fee38beea70>, <datasets.replica.Datum

val: [<datasets.replica.Datum_feature object at 0x7fee38bf1f90>, <datasets.replica.Datum_feature object at 0x7fee38bf1ff0>, <datasets.replica.Datum_feature object at 0x7fee38bf2050>, <datasets.replica.Datum_feature object at 0x7fee38bf20b0>, <datasets.replica.Datum_feature object at 0x7fee38bf2110>, <datasets.replica.Datum_feature object at 0x7fee38bf2170>, <datasets.replica.Datum_feature object at 0x7fee38bf21d0>, <datasets.replica.Datum_feature object at 0x7fee38bf2230>, <datasets.replica.Datum_feature object at 0x7fee38bf2290>, <datasets.replica.Datum_feature object at 0x7fee38bf22f0>, <datasets.replica.Datum_feature object at 0x7fee38bf2350>, <datasets.replica.Datum_feature object at 0x7fee38bf23b0>, <datasets.replica.Datum_feature object at 0x7fee38bf2410>, <datasets.replica.Datum_feature object at 0x7fee38bf2470>, <datasets.replica.Datum_feature object at 0x7fee38bf24d0>, <datasets.replica.Datum_feature object at 0x7fee38bf2530>, <datasets.replica.Datum_feature object at 0x7fee38

-----------------------------------------------------------
Label range: 0-47
Unique labels: Counter({45: 18, 22: 16, 17: 14, 5: 11, 38: 11, 13: 10, 46: 10, 44: 7, 6: 7, 3: 6, 25: 6, 4: 5, 35: 5, 36: 5, 20: 4, 27: 4, 37: 4, 7: 3, 40: 3, 19: 2, 28: 2, 21: 2, 30: 2, 31: 2, 47: 2, 10: 2, 14: 2, 39: 2, 41: 2, 15: 1, 18: 1, 23: 1, 0: 1, 11: 1, 12: 1, 26: 1, 32: 1, 43: 1})
Label range: 0-47
Unique labels: Counter({22: 12, 13: 10, 46: 9, 5: 8, 28: 7, 45: 7, 37: 4, 44: 4, 17: 4, 6: 3, 29: 3, 38: 3, 4: 2, 20: 2, 24: 2, 25: 2, 2: 2, 3: 2, 7: 2, 39: 2, 0: 1, 1: 1, 11: 1, 16: 1, 21: 1, 26: 1, 31: 1, 43: 1, 47: 1, 8: 1, 10: 1, 14: 1, 19: 1, 35: 1, 40: 1, 42: 1})
Label range: 2-47
Unique labels: Counter({13: 18, 22: 8, 45: 7, 5: 6, 44: 5, 46: 5, 8: 5, 43: 5, 2: 2, 3: 2, 20: 2, 25: 2, 38: 2, 21: 2, 31: 2, 33: 2, 10: 1, 14: 1, 41: 1, 7: 1, 9: 1, 34: 1, 37: 1, 47: 1})
Building transform_train
+ random resized crop (size=(224, 224), scale=(0.08, 1.0))
+ random flip
+ to torch tensor of range [0, 1]
+ no

100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


output: tensor([[ 4.4688,  5.8164,  4.8086,  ...,  7.3281,  7.4805,  4.0977],
        [ 5.5742,  6.9219,  5.9414,  ...,  8.4219,  8.5781,  5.1641],
        [ 4.9922,  6.0742,  5.3750,  ...,  7.4219,  7.5273,  4.6953],
        ...,
        [ 7.1680,  8.0547,  7.3828,  ...,  9.9453,  9.5000,  6.5625],
        [ 8.5078,  9.3906,  8.7109,  ..., 11.3203, 10.9062,  7.8828],
        [ 2.6543,  4.1367,  2.7383,  ...,  4.8281,  5.0078,  2.5039]],
       device='cuda:0', dtype=torch.float16)
label: tensor([ 2,  2,  3,  3, 10, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 20, 22,
        22, 22, 22, 25, 25, 38, 41, 44, 45, 45, 45, 46, 46,  5,  5,  5,  5,  5,
         5,  7,  8,  8,  8,  8,  8,  9, 13, 13, 13, 13, 13, 13, 13, 13, 20, 21,
        21, 22, 22, 22, 22, 31, 31, 33, 33, 34, 37, 38, 43, 43, 43, 43, 43, 44,
        44, 44, 44, 45, 45, 45, 45, 46, 46, 46, 47], device='cuda:0')
=> result
* total: 83
* correct: 26
* accuracy: 31.3%
* error: 68.7%
* macro_f1: 5.6%
